In [1]:
# This is how the triagle lattice data is generated. You may find it helpful to generate some 
# of your own data
from __future__ import division
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt

import jax.numpy as jnp
from jax import jit, vmap

import tensorflow as tf
import tensorflow.keras as keras


from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import optimizers


class Ising_tri():
    ''' Simulating the Ising model '''  
    def __init__(self, size, temp):
        self.temp = temp
        self.N = int(size)
    ## monte carlo moves
    def mcmove(self, config, N, beta):
        ''' This is to execute the monte carlo moves using 
        Metropolis algorithm such that detailed
        balance condition is satisified'''
        for i in range(N):
            for j in range(N):            
                    a = np.random.randint(0, N) # select a row
                    b = np.random.randint(0, N) # select a column
                    s =  config[a, b] # current state at (a, b)
                    if a%2:
                        nb = config[(a+1)%N,b] +config[(a+1)%N,(b+1)%N] + config[a,(b+1)%N] + \
                        config[(a-1)%N,b] + config[(a-1)%N,(b+1)%N] + config[a,(b-1)%N]
                    else:
                        nb = config[(a+1)%N,b] +config[(a+1)%N,(b-1)%N] + config[a,(b+1)%N] + \
                        config[(a-1)%N,b] + config[(a-1)%N,(b-1)%N] + config[a,(b-1)%N]
                    
                    
                    cost = 2*s*nb
                    if cost < 0:	
                        s *= -1
                    elif rand() < np.exp(-cost*beta):
                        s *= -1
                    config[a, b] = s
        return config
    
    def simulate(self):   
        ''' This module simulates the Ising model'''
        config = 2*np.random.randint(2, size=(self.N,self.N))-1   
        msrmnt = 81
        for i in range(msrmnt):
            self.mcmove(config, self.N, 1.0/self.temp)
        return config

Using TensorFlow backend.


You can import 4-temp data for square and triangular lattices as follows

In [2]:
N = 250
nx, ny = 32, 32

Xsq = np.ndarray((4*N,nx,ny,1))
ysq = np.ndarray(4*N)

for i in np.arange(N):
    Xsq[i + 0*N] = np.loadtxt("./square_T1/square_T1/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ysq[i + 0*N] = 0
    Xsq[i + 1*N] = np.loadtxt("./square_T2/square_T2/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ysq[i + 1*N] = 1
    Xsq[i + 2*N] = np.loadtxt("./square_T3/square_T3/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ysq[i + 2*N] = 2
    Xsq[i + 3*N] = np.loadtxt("./square_T4/square_T4/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ysq[i + 3*N] = 3

Xsq_train, Xsq_test, ysq_train, ysq_test = train_test_split(Xsq, ysq, test_size=0.2, random_state=0)

In [3]:
N = 250
nx, ny = 32, 32

Xtri = np.ndarray((4*N,nx,ny,1))
ytri = np.ndarray(4*N)

for i in np.arange(N):
    Xtri[i + 0*N] = np.loadtxt("./triangle_T1/triangle_T1/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ytri[i + 0*N] = 0
    Xtri[i + 1*N] = np.loadtxt("./triangle_T2/triangle_T2/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ytri[i + 1*N] = 1
    Xtri[i + 2*N] = np.loadtxt("./triangle_T3/triangle_T3/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ytri[i + 2*N] = 2
    Xtri[i + 3*N] = np.loadtxt("./triangle_T4/triangle_T4/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ytri[i + 3*N] = 3

Xtri_train, Xtri_test, ytri_train, ytri_test = train_test_split(Xtri, ytri, test_size=0.2, random_state=0)

Make sure you know the shape of data.

In [4]:
print("Shape of training data:")
print(Xsq_train.shape, Xtri_train.shape)
print(ysq_train.shape, ytri_train.shape)
print("Shape of test data:")
print(Xsq_test.shape, Xtri_test.shape)
print(ysq_test.shape, ytri_test.shape)

Shape of training data:
(800, 32, 32, 1) (800, 32, 32, 1)
(800,) (800,)
Shape of test data:
(200, 32, 32, 1) (200, 32, 32, 1)
(200,) (200,)


### (a) Train a fully connected neural network to do the classification on both datasets. Then, train  a  convolutional  neural  network  to  do  the  classification,  on  both datasets.   Make  a  table  of  your  performance  numbers  for  both  models  and  upload  these  numbers.   This,  together  with  your code,  should be uploaded to the course website when you turn in your homework.

The temperatures for square lattice are $T = 1.5, 2.1, 2.4, 3.5$. $T = 2.5, 3.2, 3.8, 5$ for triangle lattice.


Solution to (a):

In [5]:
class small_FNN:
    def __init__(self):
        model = self
    
    @staticmethod
    def build(width, height, depth, num_classes, channels_first=False):
        model = Sequential()

        if channels_first == False:
            inputShape = (height, width, depth)
            chanDim = -1
        elif channels_first == True:
            inputShape = (depth, height, width)
            chanDim = 1
            
        model.add(Flatten())
        
        model.add(Dense(256,  activation='relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        
        model.add(Dense(128,  activation='relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        
        model.add(Dense(16, activation='relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        
        model.add(Dense(num_classes, activation="softmax"))
        
        return model

In [8]:
def train_model(model, train_data, train_lbls, test_data, 
                test_lbls, num_classes, input_shape, hyperparams):
    # Ensure data is shaped properly, assumes channels last set up
    x_train = train_data
    x_test = test_data
    
    # Create categorical labels
    y_train = keras.utils.to_categorical(train_lbls, num_classes)
    y_test = keras.utils.to_categorical(test_lbls, num_classes)
     
    # Set hyperparameters
    INIT_LR = FNN_hyperparams[0]# learning rate
    EPOCHS = FNN_hyperparams[1] # number of epochs
    BS = FNN_hyperparams[2] # batch size
    OPT = optimizers.Adagrad(lr=INIT_LR) # optimizing function
    
    # Compile the model
    model.compile(loss='categorical_crossentropy',optimizer=OPT,metrics=['accuracy'])
    
    
    H = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=EPOCHS,
                  batch_size=BS)
    
    return H, model
    

In [14]:
input_shape = (32, 32, 1)
num_classes = 4
fnn_model = small_FNN.build(width=input_shape[0], height=input_shape[1], depth=input_shape[2],
                   num_classes=num_classes)

In [15]:
# hyperparams should be a tuple of: INIT_LR, EPOCHS, BS
FNN_hyperparams = (0.01, 50, 64)
H_sq_FNN, sq_FNN_model = train_model(fnn_model, Xsq_train, ysq_train, Xsq_test, ysq_test, num_classes, input_shape, FNN_hyperparams)
H_tri_FNN, tri_FNN_model = train_model(fnn_model, Xtri_train, ytri_train, Xtri_test, ytri_test, num_classes, input_shape, FNN_hyperparams)

Train on 800 samples, validate on 200 samples
Epoch 1/50
800/800 [==============================] - 1s 1ms/step - loss: 1.5823 - accuracy: 0.3225 - val_loss: 2.0615 - val_accuracy: 0.3000
Epoch 2/50
800/800 [==============================] - 0s 104us/step - loss: 1.3103 - accuracy: 0.4250 - val_loss: 1.9946 - val_accuracy: 0.2800
Epoch 3/50
800/800 [==============================] - 0s 99us/step - loss: 1.1640 - accuracy: 0.4938 - val_loss: 1.8652 - val_accuracy: 0.1900
Epoch 4/50
800/800 [==============================] - 0s 95us/step - loss: 1.0206 - accuracy: 0.5713 - val_loss: 1.7707 - val_accuracy: 0.2100
Epoch 5/50
800/800 [==============================] - 0s 101us/step - loss: 1.0204 - accuracy: 0.5550 - val_loss: 1.6554 - val_accuracy: 0.2400
Epoch 6/50
800/800 [==============================] - 0s 88us/step - loss: 0.9622 - accuracy: 0.5850 - val_loss: 1.5808 - val_accuracy: 0.2550
Epoch 7/50
800/800 [==============================] - 0s 98us/step - loss: 0.8700 - accuracy: 0

800/800 [==============================] - 0s 83us/step - loss: 0.5398 - accuracy: 0.7613 - val_loss: 0.9676 - val_accuracy: 0.5600
Epoch 8/50
800/800 [==============================] - 0s 93us/step - loss: 0.5019 - accuracy: 0.7875 - val_loss: 0.9441 - val_accuracy: 0.5950
Epoch 9/50
800/800 [==============================] - 0s 86us/step - loss: 0.4522 - accuracy: 0.8100 - val_loss: 0.7951 - val_accuracy: 0.6050
Epoch 10/50
800/800 [==============================] - 0s 89us/step - loss: 0.4449 - accuracy: 0.7975 - val_loss: 0.7304 - val_accuracy: 0.6200
Epoch 11/50
800/800 [==============================] - 0s 82us/step - loss: 0.4354 - accuracy: 0.8225 - val_loss: 0.6667 - val_accuracy: 0.6750
Epoch 12/50
800/800 [==============================] - 0s 83us/step - loss: 0.4146 - accuracy: 0.8288 - val_loss: 0.5989 - val_accuracy: 0.6650
Epoch 13/50
800/800 [==============================] - 0s 85us/step - loss: 0.3992 - accuracy: 0.8350 - val_loss: 0.6157 - val_accuracy: 0.6600
Epoch 

### (b) Train a convolutional neural network to do the classification, on both datasets. Make a table of your performance numbers for (a) and (b). 
Try to optimize the performance of your models and compare the result.

solution to (b):

In [16]:
class SmallVGGNet:
    def __init__(self):
        model = self
    
    @staticmethod
    def build(width, height, depth, num_classes, channels_first=False):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        
        if channels_first == False:
            inputShape = (height, width, depth)
            chanDim = -1
        elif channels_first == True:
            inputShape = (depth, height, width)
            chanDim = 1

        # CONV => RELU => POOL layer set
        model.add(Conv2D(32, (3, 3), padding="same",
            input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        # (CONV => RELU) * 2 => POOL layer set
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        # (CONV => RELU) * 3 => POOL layer set
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(256))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        # softmax classifier
        model.add(Dense(num_classes))
        model.add(Activation("softmax"))

        # return the constructed network architecture
        return model
    
class SimpleCNN:
    def __init__(self):
        model = self
    
    @staticmethod
    def build(width, height, depth, num_classes, channels_first=False):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        
        if channels_first == False:
            inputShape = (height, width, depth)
            chanDim = -1
        elif channels_first == True:
            inputShape = (depth, height, width)
            chanDim = 1
            
        model.add(Conv2D(32, (5, 5), activation='relu', input_shape=inputShape))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        #model.add(Dropout(0.25))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        #model.add(MaxPooling2D((2, 2)))
        #model.add(BatchNormalization(axis=chanDim))
                  
        model.add(Flatten())
        model.add(Dropout(0.25))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(num_classes))
        model.add(Activation("softmax"))

        # return the constructed network architecture
        return model

In [18]:
input_shape = (32, 32, 1)
num_classes = 4
cnn_model = SimpleCNN.build(width=input_shape[0], height=input_shape[1], depth=input_shape[2],
                   num_classes=num_classes)

In [20]:
CNN_hyperparams = (0.001, 50, 64)
H_sq_CNN, sq_CNN_model = train_model(cnn_model, Xsq_train, ysq_train, Xsq_test, ysq_test, num_classes, input_shape, CNN_hyperparams)
H_tri_CNN, tri_CNN_model = train_model(cnn_model, Xtri_train, ytri_train, Xtri_test, ytri_test, num_classes, input_shape, CNN_hyperparams)

Train on 800 samples, validate on 200 samples
Epoch 1/50
800/800 [==============================] - 6s 8ms/step - loss: 1.5719 - accuracy: 0.3175 - val_loss: 1.2507 - val_accuracy: 0.3550
Epoch 2/50
800/800 [==============================] - 0s 91us/step - loss: 1.0212 - accuracy: 0.5200 - val_loss: 1.2569 - val_accuracy: 0.3600
Epoch 3/50
800/800 [==============================] - 0s 83us/step - loss: 0.8173 - accuracy: 0.6388 - val_loss: 0.8935 - val_accuracy: 0.6100
Epoch 4/50
800/800 [==============================] - 0s 79us/step - loss: 0.6023 - accuracy: 0.7425 - val_loss: 1.2674 - val_accuracy: 0.4100
Epoch 5/50
800/800 [==============================] - 0s 78us/step - loss: 0.5108 - accuracy: 0.8000 - val_loss: 0.3378 - val_accuracy: 0.8850
Epoch 6/50
800/800 [==============================] - 0s 82us/step - loss: 0.4624 - accuracy: 0.8112 - val_loss: 0.3110 - val_accuracy: 0.9100
Epoch 7/50
800/800 [==============================] - 0s 82us/step - loss: 0.3457 - accuracy: 0.8

800/800 [==============================] - 0s 89us/step - loss: 0.1354 - accuracy: 0.9650 - val_loss: 0.1320 - val_accuracy: 0.9650
Epoch 8/50
800/800 [==============================] - 0s 85us/step - loss: 0.1533 - accuracy: 0.9438 - val_loss: 0.1437 - val_accuracy: 0.9650
Epoch 9/50
800/800 [==============================] - 0s 86us/step - loss: 0.1230 - accuracy: 0.9663 - val_loss: 0.1458 - val_accuracy: 0.9450
Epoch 10/50
800/800 [==============================] - 0s 79us/step - loss: 0.0965 - accuracy: 0.9725 - val_loss: 0.2995 - val_accuracy: 0.9100
Epoch 11/50
800/800 [==============================] - 0s 81us/step - loss: 0.0884 - accuracy: 0.9762 - val_loss: 0.1400 - val_accuracy: 0.9550
Epoch 12/50
800/800 [==============================] - 0s 83us/step - loss: 0.0833 - accuracy: 0.9725 - val_loss: 0.1278 - val_accuracy: 0.9550
Epoch 13/50
800/800 [==============================] - 0s 81us/step - loss: 0.0747 - accuracy: 0.9762 - val_loss: 0.1423 - val_accuracy: 0.9550
Epoch 

### (c) We have provided a test set of 10 spins configurations for each of the two problems. Each of the spin configurations is not necessarily at the temperatures of the training sets. Calculate your best estimate of the temperatures of these spin configuration. Upload your results to Kaggle.
[Hint: A direct fingerprint of temperature is the distribution of spin up
and down, because you can image that the spins fluctuate more violently
at higher temperature. Although the mothod you use in homework 2 can also work, you may be interested in trying to take distribution into account when you
build the model to estimate temperature and see if you can make use of this extra information. This may help you win the
kaggle. It is totally fine if you find that the information of distribution is not helpful. Note also that a CNN kind-of does this. One possibility is that you may want a CNN that captures enough distribution information.]

Solution to (c)

### (d) *Transfer Learning*.  
As we emphasize in class, one can freeze the training of the bottom layers of a network and retrain the top part of the network to adopt to a new situation. Use your CNN that you trained on the squarelattice data to do transfer learning on the triangular lattice data.  How does the performance compare to that of the direct methods?  Add the performance numbers for transfer learning in your table from Part (a). Note that the training time and number of training examples needed for transfer learning is far less than that for the direct  optimization. For  example,  is  50  triangle  example  sufficient  for the re-training process?  Use your transfer learning result to predict the transition temperature of triangle lattice Ising model, as demonstrated in this [Nature Physics](https://www-nature-com.ezp-prod1.hul.harvard.edu/articles/nphys4035.pdf) publication.

As a guideline, you may like to just change the last `Dense` layer with `softmax` activation when you do the transfer learning. Other choices are also OK.

Solution to (d):

In [21]:
sq_CNN_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten_3 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 1024)             

In [25]:
sq_CNN_model.pop() # this removes the last layer

AttributeError: 'NoneType' object has no attribute 'pop'

In [24]:
sq_CNN_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten_3 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 1024)             